# Capacidad Hospitalaria en Europa y envejecimiento de la poblacion: foco en España

In [92]:
import pandas as pd 
import numpy as np 
import requests
import data_utils as du

## Cargando tablas

In [93]:
#Descargadas de la pagina Eurostat
df_bed_hosp = pd.read_csv('tps00046_page_linear.csv.gz')
df_hospitalarias_estancia = pd.read_csv('hospitalaria_estancia.tsv.gz', sep='\t')
df_gasto_sanitario = pd.read_csv('gasto_sanitario.tsv.gz', sep='\t')

## Cargando JSON

In [94]:
#Api del Instituto Nacional de Estadistica
url = "https://servicios.ine.es/wstempus/js/ES/DATOS_TABLA/56934"
response = requests.get(url)


In [95]:
response.reason

'OK'

In [96]:
response.status_code

200

## Limpieza de la API 

In [97]:
data = response.json()
df_population = pd.DataFrame(data)
df_population.head()

,COD,Nombre,FK_Unidad,FK_Escala,Notas,Data
0,ECP320,Total Nacional. Todas las edades. Total. Pobla...,3,1,[{'texto': 'https://www.ine.es/dyngs/INEbase/e...,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."
1,ECP319,Total Nacional. Todas las edades. Hombres. Pob...,3,1,NaN,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."
2,ECP318,Total Nacional. Todas las edades. Mujeres. Pob...,3,1,NaN,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."
3,ECP317,Total Nacional. 0 años. Total. Población. Núme...,3,1,NaN,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."
4,ECP316,Total Nacional. 0 años. Hombres. Población. Nú...,3,1,NaN,"[{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'F..."


In [98]:
df_long = df_population.explode("Data").reset_index(drop=True)

In [99]:
df_long

,COD,Nombre,FK_Unidad,FK_Escala,Notas,Data
0,ECP320,Total Nacional. Todas las edades. Total. Pobla...,3,1,[{'texto': 'https://www.ine.es/dyngs/INEbase/e...,"{'Fecha': 1735686000000, 'FK_TipoDato': 1, 'FK..."
1,ECP320,Total Nacional. Todas las edades. Total. Pobla...,3,1,[{'texto': 'https://www.ine.es/dyngs/INEbase/e...,"{'Fecha': 1727733600000, 'FK_TipoDato': 1, 'FK..."
2,ECP320,Total Nacional. Todas las edades. Total. Pobla...,3,1,[{'texto': 'https://www.ine.es/dyngs/INEbase/e...,"{'Fecha': 1719784800000, 'FK_TipoDato': 1, 'FK..."
3,ECP320,Total Nacional. Todas las edades. Total. Pobla...,3,1,[{'texto': 'https://www.ine.es/dyngs/INEbase/e...,"{'Fecha': 1711922400000, 'FK_TipoDato': 1, 'FK..."
4,ECP320,Total Nacional. Todas las edades. Total. Pobla...,3,1,[{'texto': 'https://www.ine.es/dyngs/INEbase/e...,"{'Fecha': 1704063600000, 'FK_TipoDato': 1, 'FK..."
...,...,...,...,...,...,...
36115,ECP0,Total Nacional. 105 y más años. Mujeres. Pobla...,3,1,NaN,"{'Fecha': 1404165600000, 'FK_TipoDato': 1, 'FK..."
36116,ECP0,Total Nacional. 105 y más años. Mujeres. Pobla...,3,1,NaN,"{'Fecha': 1388530800000, 'FK_TipoDato': 1, 'FK..."
36117,ECP0,Total Nacional. 105 y más años. Mujeres. Pobla...,3,1,NaN,"{'Fecha': 1372629600000, 'FK_TipoDato': 1, 'FK..."
36118,ECP0,Total Nacional. 105 y más años. Mujeres. Pobla...,3,1,NaN,"{'Fecha': 1356994800000, 'FK_TipoDato': 1, 'FK..."


In [100]:
df_long["year"] = df_long["Data"].apply(lambda x: x["Anyo"])
df_long["population"] = df_long["Data"].apply(lambda x: x["Valor"])

In [101]:
def extract_sex(nombre):
    if "Hombres" in nombre:
        return "male"
    if "Mujeres" in nombre:
        return "female"
    return "total"

df_long["sex"] = df_long["Nombre"].apply(extract_sex)

In [102]:
import re

def extract_age(nombre):
    if "Todas las edades" in nombre:
        return "all"
    m = re.search(r"(\d+)\s+años", nombre)
    if m:
        return int(m.group(1))
    if "y más años" in nombre:
        m = re.search(r"(\d+)", nombre)
        if m:
            return f"{m.group(1)}+"
    return None

df_long["age"] = df_long["Nombre"].apply(extract_age)

In [103]:
df_pop = df_long[["year", "age", "sex", "population"]]

In [104]:
df_pop

,year,age,sex,population
0,2025,all,total,49128297.0
1,2024,all,total,48999880.0
2,2024,all,total,48821936.0
3,2024,all,total,48701130.0
4,2024,all,total,48619695.0
...,...,...,...,...
36115,2014,105+,female,534.0
36116,2014,105+,female,581.0
36117,2013,105+,female,602.0
36118,2013,105+,female,683.0


In [105]:
df_pop_sorted = df_pop.sort_values(["year"], ascending=False)

df_pop_year = (
    df_pop_sorted
    .groupby(["year", "age", "sex"], as_index=False)
    .first()
)

In [106]:
import numpy as np

def age_to_num(x):
    if x == "all":
        return np.nan
    if isinstance(x, str) and "+" in x:
        return int(x.replace("+", ""))
    return x

df_pop_year["age_num"] = df_pop_year["age"].apply(age_to_num)

In [107]:
df_pop_year

,year,age,sex,population,age_num
0,1971,0,female,314740.0,0.0
1,1971,0,male,334877.0,0.0
2,1971,0,total,647443.0,0.0
3,1971,2,female,308660.0,2.0
4,1971,2,male,324917.0,2.0
...,...,...,...,...,...
16747,2025,85+,male,528961.0,85.0
16748,2025,85+,total,1566850.0,85.0
16749,2025,all,female,25037928.0,NaN
16750,2025,all,male,24090369.0,NaN


In [108]:
pop_total = df_pop_year[
    (df_pop_year["age"] == "all") &
    (df_pop_year["sex"] == "total")
][["year", "population"]].rename(columns={"population": "pop_total"})

In [109]:
pop_65 = (
    df_pop_year[
        (df_pop_year["sex"] == "total") &
        (df_pop_year["age_num"] >= 65)
    ]
    .groupby("year", as_index=False)["population"]
    .sum()
    .rename(columns={"population": "pop_65"})
)

In [110]:
pop_65

,year,pop_65
0,1971,3288136.0
1,1972,3341575.0
2,1973,3452071.0
3,1974,3571638.0
4,1975,3616290.0
5,1976,3745923.0
6,1977,3839150.0
7,1978,3975840.0
8,1979,4055568.0
9,1980,4155148.0


In [111]:
df_demo = pop_total.merge(pop_65, on="year")

df_demo["pct_65_plus"] = df_demo["pop_65"] / df_demo["pop_total"] * 100

In [112]:
df_demo

,year,pop_total,pop_65,pct_65_plus
0,1971,34040642.0,3288136.0,9.659442
1,1972,34595886.0,3341575.0,9.658880
2,1973,34980317.0,3452071.0,9.868610
3,1974,35363890.0,3571638.0,10.099675
4,1975,35569375.0,3616290.0,10.166864
5,1976,35946425.0,3745923.0,10.420850
6,1977,36506811.0,3839150.0,10.516257
7,1978,36694077.0,3975840.0,10.835100
8,1979,37035719.0,4055568.0,10.950423
9,1980,37346940.0,4155148.0,11.125806


In [89]:
pop_total = df_pop_year[
    (df_pop_year["age"] == "all") &
    (df_pop_year["sex"] == "total")
]

In [90]:
pop_65 = df_pop_year[
    (df_pop_year["sex"] == "total") &
    (df_pop_year["age"].apply(lambda x: isinstance(x, int) and x >= 65))
].groupby("year", as_index=False)["population"].sum()

In [91]:
pop_total

,year,age,sex,population
257,1971,all,total,34040642.0
515,1972,all,total,34595886.0
773,1973,all,total,34980317.0
1031,1974,all,total,35363890.0
1289,1975,all,total,35569375.0
1547,1976,all,total,35946425.0
1805,1977,all,total,36506811.0
2063,1978,all,total,36694077.0
2321,1979,all,total,37035719.0
2579,1980,all,total,37346940.0


In [75]:
# voy a hacer esto porque los datos completos aparecen en la columna data al abrir la api y la primera fila que contiene todos los datos
texto = df_population.loc[0, "Nombre"] # aqui selcciono la primera fila con todo el textou

mask = df_population["Nombre"].str.contains(
    texto,
    na=False
)

total_row = df_population[mask].iloc[0]
df_pop_total = pd.DataFrame(total_row["Data"])
df_pop_total.head()

,Fecha,FK_TipoDato,FK_Periodo,Anyo,Valor,Secreto
0,1735686000000,1,19,2025,49128297.0,False
1,1727733600000,1,22,2024,48999880.0,False
2,1719784800000,1,21,2024,48821936.0,False
3,1711922400000,1,20,2024,48701130.0,False
4,1704063600000,1,19,2024,48619695.0,False


In [76]:
#arreglare la columna fecha porque los datos en un año son trismestrales y quiero quedarme con la mas reciente de ese año
df_pop_total['Fecha'] = pd.to_datetime(df_pop_total['Fecha'], unit='ms')
df_pop_total = df_pop_total.sort_values("Fecha")

In [77]:
# Y ahora solo quiero quedarme con el ultimo valor de ese año
df_pop_total = df_pop_total.groupby("Anyo").tail(1)


In [78]:
#Renombrare las columnas con las que voy a trabajar que serian 
df_pop_total = df_pop_total.rename(columns={
    "Anyo" : "year",
    "Valor" : "population"
})

In [11]:
df_pop_total = df_pop_total[['year', 'population']]

In [12]:
#me quedo con los datos despues del 2000, porque no siempre estan los datos completos antes del 2000
#muchos paises entran mas tarde en la union europea 
df_pop_total = df_pop_total[df_pop_total["year"] >= 2000]

## Limpieza de la tabla bed_hospital

In [13]:
du.show_total_info(df_bed_hosp,2)

HEAD: 


,DATAFLOW,LAST UPDATE,freq,unit,facility,hlthcare,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:TPS00046(1.0),15/07/25 11:00:00,Annual,Per hundred thousand inhabitants,Available beds in hospitals (HP.1),Total,Albania,2013,288.97,NaN,NaN
1,ESTAT:TPS00046(1.0),15/07/25 11:00:00,Annual,Per hundred thousand inhabitants,Available beds in hospitals (HP.1),Total,Austria,2013,731.76,NaN,NaN



INFO: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATAFLOW     404 non-null    object 
 1   LAST UPDATE  404 non-null    object 
 2   freq         404 non-null    object 
 3   unit         404 non-null    object 
 4   facility     404 non-null    object 
 5   hlthcare     404 non-null    object 
 6   geo          404 non-null    object 
 7   TIME_PERIOD  404 non-null    int64  
 8   OBS_VALUE    404 non-null    float64
 9   OBS_FLAG     46 non-null     object 
 10  CONF_STATUS  0 non-null      float64
dtypes: float64(2), int64(1), object(8)
memory usage: 34.8+ KB

DESCRIBE: 


,TIME_PERIOD,OBS_VALUE,CONF_STATUS
count,404.000000,404.000000,0.0
mean,2018.034653,472.819629,NaN
std,3.173836,170.877254,NaN
min,2013.000000,101.620000,NaN
25%,2015.000000,321.802500,NaN
50%,2018.000000,446.480000,NaN
75%,2021.000000,600.797500,NaN
max,2024.000000,864.390000,NaN



SHAPE: 
(404, 11)


In [14]:
#unidades beds per hundred thousand inhabitants (100.000 habitantes)
df_bed_hosp.head()

,DATAFLOW,LAST UPDATE,freq,unit,facility,hlthcare,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:TPS00046(1.0),15/07/25 11:00:00,Annual,Per hundred thousand inhabitants,Available beds in hospitals (HP.1),Total,Albania,2013,288.97,NaN,NaN
1,ESTAT:TPS00046(1.0),15/07/25 11:00:00,Annual,Per hundred thousand inhabitants,Available beds in hospitals (HP.1),Total,Austria,2013,731.76,NaN,NaN
2,ESTAT:TPS00046(1.0),15/07/25 11:00:00,Annual,Per hundred thousand inhabitants,Available beds in hospitals (HP.1),Total,Austria,2014,731.65,NaN,NaN
3,ESTAT:TPS00046(1.0),15/07/25 11:00:00,Annual,Per hundred thousand inhabitants,Available beds in hospitals (HP.1),Total,Austria,2015,735.29,NaN,NaN
4,ESTAT:TPS00046(1.0),15/07/25 11:00:00,Annual,Per hundred thousand inhabitants,Available beds in hospitals (HP.1),Total,Austria,2016,731.91,NaN,NaN


In [15]:
# me quedare solo con geo -> que me da el pais(UE), TIME_PERIOD -> año de referencia, y OBS_VALUE -> numero de camas por cada 100.000 habitantes
df_bed_hosp = df_bed_hosp[[
    'geo', 
    'TIME_PERIOD',
    'OBS_VALUE'
]]

In [16]:
#cambiando nombre de columnas, para mejorar el entendimiento de los valores 
df_bed_hosp = df_bed_hosp.rename(columns= {
    'geo': 'country',
    "TIME_PERIOD": "year",
    "OBS_VALUE": "beds_per_100k"
})

In [17]:
du.show_total_info(df_bed_hosp, 2)

HEAD: 


,country,year,beds_per_100k
0,Albania,2013,288.97
1,Austria,2013,731.76



INFO: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   country        404 non-null    object 
 1   year           404 non-null    int64  
 2   beds_per_100k  404 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 9.6+ KB

DESCRIBE: 


,year,beds_per_100k
count,404.000000,404.000000
mean,2018.034653,472.819629
std,3.173836,170.877254
min,2013.000000,101.620000
25%,2015.000000,321.802500
50%,2018.000000,446.480000
75%,2021.000000,600.797500
max,2024.000000,864.390000



SHAPE: 
(404, 3)


In [18]:
#cambiando tipos de datos a columna de beds_per_100k
df_bed_hosp["beds_per_100k"] = pd.to_numeric(df_bed_hosp["beds_per_100k"], errors="coerce")

In [19]:
df_bed_hosp

,country,year,beds_per_100k
0,Albania,2013,288.97
1,Austria,2013,731.76
2,Austria,2014,731.65
3,Austria,2015,735.29
4,Austria,2016,731.91
...,...,...,...
399,Türkiye,2019,287.61
400,Türkiye,2020,301.23
401,Türkiye,2021,302.44
402,Türkiye,2022,308.53


In [20]:
df_bed_hosp['country'].value_counts()

country
Belgium                                      12
Italy                                        11
Norway                                       11
Lithuania                                    11
Luxembourg                                   11
Latvia                                       11
Montenegro                                   11
Malta                                        11
Netherlands                                  11
Poland                                       11
Austria                                      11
Portugal                                     11
Romania                                      11
Serbia                                       11
Sweden                                       11
Slovenia                                     11
Slovakia                                     11
Liechtenstein                                11
Türkiye                                      11
Iceland                                      11
Ireland                         

### Limpieza tabla estancia hospitalarias

In [21]:
#viendo los nombres de columnas, vamos a arreglarlo 
df_hospitalarias_estancia.columns

Index(['freq,hlthcare,indic_he,unit,geo\TIME_PERIOD', '1960 ', '1961 ',
       '1962 ', '1963 ', '1964 ', '1965 ', '1966 ', '1967 ', '1968 ', '1969 ',
       '1970 ', '1971 ', '1972 ', '1973 ', '1974 ', '1975 ', '1976 ', '1977 ',
       '1978 ', '1979 ', '1980 ', '1981 ', '1982 ', '1983 ', '1984 ', '1985 ',
       '1986 ', '1987 ', '1988 ', '1989 ', '1990 ', '1991 ', '1992 ', '1993 ',
       '1994 ', '1995 ', '1996 ', '1997 ', '1998 ', '1999 ', '2000 ', '2001 ',
       '2002 ', '2003 ', '2004 ', '2005 ', '2006 ', '2007 ', '2008 ', '2009 ',
       '2010 ', '2011 ', '2012 ', '2013 ', '2014 ', '2015 ', '2016 ', '2017 ',
       '2018 ', '2019 ', '2020 ', '2021 ', '2022 ', '2023 ', '2024 '],
      dtype='object')

In [22]:
# haciendo una copia, para ver como arreglar esto
df = df_hospitalarias_estancia.copy()

# separar la columna mal formada
first_col = df_hospitalarias_estancia.columns[0]

df[first_col].head()

0    A,PSY,ALOS,NR,AT
1    A,PSY,ALOS,NR,BE
2    A,PSY,ALOS,NR,BG
3    A,PSY,ALOS,NR,CH
4    A,PSY,ALOS,NR,CY
Name: freq,hlthcare,indic_he,unit,geo\TIME_PERIOD, dtype: object

In [23]:
first_col = df.columns[0]

#vamos separando loas columnas
split_cols = df[first_col].str.split(",", expand=True)
split_cols.columns = ["freq", "hlthcare", "indic_he", "unit", "geo"]

df = pd.concat(
    [split_cols, df.drop(columns=[first_col])],
    axis=1
)

In [24]:
df

,freq,hlthcare,indic_he,unit,geo,1960,1961,1962,1963,1964,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,A,PSY,ALOS,NR,AT,:,:,:,:,:,...,18.54,18.29,18.50,18.18,19.65,18.63,18.90,17.96,19.69,:
1,A,PSY,ALOS,NR,BE,:,:,:,:,:,...,:,9.35 d,9.23 d,9.29 d,8.96 d,9.38 d,9.23 d,9.49 d,9.55 d,:
2,A,PSY,ALOS,NR,BG,:,:,:,:,:,...,:,:,:,:,28.90,32.60,32.70,32.30,32 b,:
3,A,PSY,ALOS,NR,CH,:,:,:,:,:,...,35.12,34.22,33.27,34.28,33.62,34.21,33.69,31.85,33.94,:
4,A,PSY,ALOS,NR,CY,:,:,:,:,:,...,:,:,:,:,23.20,24.80,28.40,31.90,33.15,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,A,TOTAL,DISCH,NR,SE,:,:,:,:,:,...,1453812.00,1419218.00,1378167.00,1373585.00,1366639.00,1286075.00,1310295.00,1291762.00,1267176.00,:
296,A,TOTAL,DISCH,NR,SI,:,:,:,:,:,...,346601.00,343299.00,332127.00,330273.00,328416.00,275693.00,286523.00,280580 b,284077.00,:
297,A,TOTAL,DISCH,NR,SK,:,:,:,:,:,...,934836.00,956694.00,944715.00,906471 b,911401.00,756925.00,737036.00,783468.00,814821.00,:
298,A,TOTAL,DISCH,NR,TR,:,:,:,:,:,...,13360507.00,13267188.00,13528778.00,13465409.00,13620087.00,10486926.00,11641364.00,13113970.00,13222278.00,:


#### ALOS (Estancia media hospitalaria)

In [25]:
#aqui hago una copia, con solo alos (en dias)
df_alos = df[df["indic_he"] == "ALOS"].copy()


In [26]:
df_alos.head()

,freq,hlthcare,indic_he,unit,geo,1960,1961,1962,1963,1964,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,A,PSY,ALOS,NR,AT,:,:,:,:,:,...,18.54,18.29,18.50,18.18,19.65,18.63,18.90,17.96,19.69,:
1,A,PSY,ALOS,NR,BE,:,:,:,:,:,...,:,9.35 d,9.23 d,9.29 d,8.96 d,9.38 d,9.23 d,9.49 d,9.55 d,:
2,A,PSY,ALOS,NR,BG,:,:,:,:,:,...,:,:,:,:,28.90,32.60,32.70,32.30,32 b,:
3,A,PSY,ALOS,NR,CH,:,:,:,:,:,...,35.12,34.22,33.27,34.28,33.62,34.21,33.69,31.85,33.94,:
4,A,PSY,ALOS,NR,CY,:,:,:,:,:,...,:,:,:,:,23.20,24.80,28.40,31.90,33.15,:


In [27]:
df_alos.columns = df_alos.columns.str.strip()
df_alos.columns

Index(['freq', 'hlthcare', 'indic_he', 'unit', 'geo', '1960', '1961', '1962',
       '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971',
       '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024'],
      dtype='object')

In [28]:
cols_years = [c for c in df_alos.columns if c.isdigit()]
len(cols_years)

65

In [29]:
df_alos_long = df_alos[["geo"] + cols_years].melt(
    id_vars="geo",
    var_name="year",
    value_name="avg_length_stay"
)

In [30]:
df_alos_long

,geo,year,avg_length_stay
0,AT,1960,:
1,BE,1960,:
2,BG,1960,:
3,CH,1960,:
4,CY,1960,:
...,...,...,...
6560,SE,2024,:
6561,SI,2024,:
6562,SK,2024,:
6563,TR,2024,:


In [31]:
df_alos_long["avg_length_stay"] = (
    df_alos_long["avg_length_stay"]
    .astype(str)
    .str.replace(":", "", regex=False)
    .str.replace(r"[a-zA-Z]", "", regex=True)
)

df_alos_long["avg_length_stay"] = pd.to_numeric(
    df_alos_long["avg_length_stay"], errors="coerce"
)


In [32]:
df_alos_long["year"] = df_alos_long["year"].astype(int)

In [33]:
df_alos_long = df_alos_long[df_alos_long["year"] >= 2013]

In [34]:
df_alos_long = df_alos_long.dropna(subset=["avg_length_stay"])

In [35]:
df_alos_long

,geo,year,avg_length_stay
5353,AT,2013,18.83
5356,CH,2013,36.92
5358,CZ,2013,11.59
5359,DE,2013,21.70
5360,EE,2013,17.70
...,...,...,...
6461,SK,2023,6.60
6462,TR,2023,4.20
6477,IS,2024,9.98
6509,IS,2024,5.57


### Tabla gasto sanitario

In [36]:
df_gasto_sanitario.head()

,"freq,unit,icha11_hc,geo\TIME_PERIOD",1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,"A,EUR_HAB,HC1,AT",:,:,:,1219.78,1251.98,1228.05,1271.26,1322.55,1361.18,...,2180.87,2252.47,2321.05,2418.12,2529.05,2585.27,2698.92,2801.9,3039.03,:
1,"A,EUR_HAB,HC1,BA",:,:,:,:,:,:,:,:,:,...,203.68,222.65,231.05,251.85,:,:,:,340.92,392.42,:
2,"A,EUR_HAB,HC1,BE",:,:,:,:,:,:,:,:,:,...,1751.78,1726.12,1808.28,1879.73,1992.23,2094.52,2173.82,2284.67,2411.31,:
3,"A,EUR_HAB,HC1,BG",:,:,:,:,:,:,:,:,:,...,245.71,264.11,290.79,317.56,344.5,418.15,480.37,509.39,593.48,:
4,"A,EUR_HAB,HC1,CH",:,:,:,:,:,:,:,:,:,...,3844.52,3821.59,3768.69,3615.78,3767.1,3824.01,3937.57,4312.67,4645.56,:


#### DISCH (Demanda hospitalaria)

In [37]:
df_disch = df[df["indic_he"] == "DISCH"].copy()

In [38]:
df_disch.columns = df_disch.columns.str.strip()

cols_years = [c for c in df_disch.columns if c.isdigit()]

In [39]:
df_disch_long = df_disch[["geo"] + cols_years].melt(
    id_vars="geo",
    var_name="year",
    value_name="hospital_discharges"
)

In [40]:
df_disch_long["hospital_discharges"] = (
    df_disch_long["hospital_discharges"]
    .astype(str)
    .str.replace(":", "", regex=False)
    .str.replace(r"[a-zA-Z]", "", regex=True)
)

df_disch_long["hospital_discharges"] = pd.to_numeric(
    df_disch_long["hospital_discharges"], errors="coerce"
)

df_disch_long["year"] = df_disch_long["year"].astype(int)

In [41]:
df_disch_long = df_disch_long[df_disch_long["year"] >= 2013]
df_disch_long = df_disch_long.dropna(subset=["hospital_discharges"])

In [42]:
df_disch_long

,geo,year,hospital_discharges
5353,AT,2013,85806.0
5356,CH,2013,70629.0
5358,CZ,2013,30631.0
5359,DE,2013,1139809.0
5360,EE,2013,12490.0
...,...,...,...
6461,SK,2023,814821.0
6462,TR,2023,13222278.0
6477,IS,2024,2647.0
6509,IS,2024,36370.0


In [43]:
df_disch_long.shape
df_disch_long.head()
df_disch_long["year"].min(), df_disch_long["year"].max()

(2013, 2024)

### Limpieza de tabla gasto sanitario

In [44]:
df_gasto_sanitario

,"freq,unit,icha11_hc,geo\TIME_PERIOD",1992,1993,1994,1995,1996,1997,1998,1999,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,"A,EUR_HAB,HC1,AT",:,:,:,1219.78,1251.98,1228.05,1271.26,1322.55,1361.18,...,2180.87,2252.47,2321.05,2418.12,2529.05,2585.27,2698.92,2801.9,3039.03,:
1,"A,EUR_HAB,HC1,BA",:,:,:,:,:,:,:,:,:,...,203.68,222.65,231.05,251.85,:,:,:,340.92,392.42,:
2,"A,EUR_HAB,HC1,BE",:,:,:,:,:,:,:,:,:,...,1751.78,1726.12,1808.28,1879.73,1992.23,2094.52,2173.82,2284.67,2411.31,:
3,"A,EUR_HAB,HC1,BG",:,:,:,:,:,:,:,:,:,...,245.71,264.11,290.79,317.56,344.5,418.15,480.37,509.39,593.48,:
4,"A,EUR_HAB,HC1,CH",:,:,:,:,:,:,:,:,:,...,3844.52,3821.59,3768.69,3615.78,3767.1,3824.01,3937.57,4312.67,4645.56,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23180,"A,PPS_HAB,TOT_HC,RS",:,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,1399.78 b,1422.21,1537.35,:
23181,"A,PPS_HAB,TOT_HC,SE",:,:,:,:,:,:,:,:,:,...,3683.88,3655.49,3697.64,3803.5,3855.18,3947.42,4245.29,4379.57,4688.07,:
23182,"A,PPS_HAB,TOT_HC,SI",:,:,:,:,:,:,:,:,:,...,1889.55,1938.87,1995.24,2124.09,2286.72,2443.23,2697.45,2955.58,3120.68,:
23183,"A,PPS_HAB,TOT_HC,SK",:,:,:,:,:,:,:,:,:,...,1508.5,1444.2,1390.76,1401.34,1519.29,1592.95,1874.62,1962.46,2088.01,:


In [45]:
###voy a separar la columna gigante
df = df_gasto_sanitario.copy()
first_col_2 = df.columns[0]
df[first_col_2].head()

0    A,EUR_HAB,HC1,AT
1    A,EUR_HAB,HC1,BA
2    A,EUR_HAB,HC1,BE
3    A,EUR_HAB,HC1,BG
4    A,EUR_HAB,HC1,CH
Name: freq,unit,icha11_hc,geo\TIME_PERIOD, dtype: object

In [46]:
split_cols = df[first_col_2].str.split(",", expand=True)
split_cols.columns = ["freq", "unit", "icha11_hc", "geo"]

df = pd.concat(
    [split_cols, df.drop(columns=[first_col_2])],
    axis=1
)

In [47]:
df.columns = df.columns.str.strip()

In [48]:
df

,freq,unit,icha11_hc,geo,1992,1993,1994,1995,1996,1997,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,A,EUR_HAB,HC1,AT,:,:,:,1219.78,1251.98,1228.05,...,2180.87,2252.47,2321.05,2418.12,2529.05,2585.27,2698.92,2801.9,3039.03,:
1,A,EUR_HAB,HC1,BA,:,:,:,:,:,:,...,203.68,222.65,231.05,251.85,:,:,:,340.92,392.42,:
2,A,EUR_HAB,HC1,BE,:,:,:,:,:,:,...,1751.78,1726.12,1808.28,1879.73,1992.23,2094.52,2173.82,2284.67,2411.31,:
3,A,EUR_HAB,HC1,BG,:,:,:,:,:,:,...,245.71,264.11,290.79,317.56,344.5,418.15,480.37,509.39,593.48,:
4,A,EUR_HAB,HC1,CH,:,:,:,:,:,:,...,3844.52,3821.59,3768.69,3615.78,3767.1,3824.01,3937.57,4312.67,4645.56,:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23180,A,PPS_HAB,TOT_HC,RS,:,:,:,:,:,:,...,:,:,:,:,:,:,1399.78 b,1422.21,1537.35,:
23181,A,PPS_HAB,TOT_HC,SE,:,:,:,:,:,:,...,3683.88,3655.49,3697.64,3803.5,3855.18,3947.42,4245.29,4379.57,4688.07,:
23182,A,PPS_HAB,TOT_HC,SI,:,:,:,:,:,:,...,1889.55,1938.87,1995.24,2124.09,2286.72,2443.23,2697.45,2955.58,3120.68,:
23183,A,PPS_HAB,TOT_HC,SK,:,:,:,:,:,:,...,1508.5,1444.2,1390.76,1401.34,1519.29,1592.95,1874.62,1962.46,2088.01,:


In [49]:
# TOT_HC = gasto total sanitario icha11_hc seleciono este porque mi hipotesis es gasto sanitario per capita vs estancia/demanda
# Total health care expenditure

# EUR_HAB = euros por habitante unit

In [50]:
#Filtrar tipo y unidad 
df = df[(df["icha11_hc"] == "TOT_HC") & (df["unit"] == "EUR_HAB")]

In [51]:
#Detectar columnas de años
df.columns = df.columns.str.strip()

cols_years = [c for c in df.columns if c.isdigit()]
len(cols_years) 

33

In [52]:
df_gasto_long = df[["geo"] + cols_years].melt(
    id_vars="geo",
    var_name="year",
    value_name="health_exp_pc"
)

In [53]:
df_gasto_long["health_exp_pc"] = (
    df_gasto_long["health_exp_pc"]
    .astype(str)
    .str.replace(":", "", regex=False)
    .str.replace(r"[a-zA-Z]", "", regex=True)
)

df_gasto_long["health_exp_pc"] = pd.to_numeric(
    df_gasto_long["health_exp_pc"], errors="coerce"
)

df_gasto_long["year"] = df_gasto_long["year"].astype(int)

In [54]:
df_gasto_long = df_gasto_long[df_gasto_long["year"] >= 2013]
df_gasto_long = df_gasto_long.dropna(subset=["health_exp_pc"])

In [55]:
df_gasto_long

,geo,year,health_exp_pc
903,AT,2013,3941.40
904,BA,2013,337.05
905,BE,2013,3761.74
906,BG,2013,419.48
907,CH,2013,6963.94
...,...,...,...
1370,RO,2023,971.52
1371,RS,2023,911.93
1372,SE,2023,5728.38
1373,SI,2023,2804.82


In [56]:
country_map = {
    # EU
    "AT": "Austria",
    "BE": "Belgium",
    "BG": "Bulgaria",
    "HR": "Croatia",
    "CY": "Cyprus",
    "CZ": "Czechia",
    "DK": "Denmark",
    "EE": "Estonia",
    "FI": "Finland",
    "FR": "France",
    "DE": "Germany",
    "EL": "Greece",
    "GR": "Greece",
    "HU": "Hungary",
    "IE": "Ireland",
    "IT": "Italy",
    "LV": "Latvia",
    "LT": "Lithuania",
    "LU": "Luxembourg",
    "MT": "Malta",
    "NL": "Netherlands",
    "PL": "Poland",
    "PT": "Portugal",
    "RO": "Romania",
    "SK": "Slovakia",
    "SI": "Slovenia",
    "ES": "Spain",
    "SE": "Sweden",

    # EFTA / Western Europe
    "CH": "Switzerland",
    "NO": "Norway",
    "IS": "Iceland",
    "LI": "Liechtenstein",
    "UK": "United Kingdom",

    # Balkans & others
    "BA": "Bosnia and Herzegovina",
    "RS": "Serbia",
    "ME": "Montenegro",
    "MK": "North Macedonia",
    "AL": "Albania",
    "XK": "Kosovo",

    # Eastern Europe / neighbours
    "UA": "Ukraine",
    "MD": "Moldova",
    "BY": "Belarus",

    # Turkey
    "TR": "Türkiye",

    # Caucasus (sometimes appear)
    "GE": "Georgia",
    "AM": "Armenia",
    "AZ": "Azerbaijan",

    # Microstates (rare but appear)
    "SM": "San Marino",
    "MC": "Monaco",
    "AD": "Andorra",
    "VA": "Vatican City"
}

In [ ]:
df_beds["country"] = df_beds["country"].map(country_map)

df_alos_long["country"] = df_alos_long["geo"].map(country_map)
df_disch_long["country"] = df_disch_long["geo"].map(country_map)
df_gasto_long["country"] = df_gasto_long["geo"].map(country_map)

In [ ]:
df_alos_long[df_alos_long["country"].isna()]["geo"].unique()

#### Tabla de Sanitaria

In [56]:
#pip install xlrd

In [ ]:
#!pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [58]:
dfs = pd.read_html("Tabla-9af0d933-634f-4431-99fb-b0e777fb51d0.xls")

len(dfs)

7

In [59]:
for i, d in enumerate(dfs):
    print("Tabla", i)
    display(d.head())

Tabla 0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...,Resumen por Comunidad Autónoma - Tipo de hospi...


Tabla 1


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026,Fecha de emisión: 02/02/2026


Tabla 2


...

,Unnamed: 0,Número de altas
0,1997,3.004.200
1,1998,3.149.124
2,1999,3.092.087
3,2000,3.238.022
4,2001,3.297.074


Tabla 5


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...,Fuente: Ministerio de Sanidad. Subdirección Ge...


Tabla 6


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...,Dirección de la web: http://pestadistico.intel...


In [60]:
df_altas_spain = dfs[4].copy()

In [61]:
df_altas_spain

,Unnamed: 0,Número de altas
0,1997,3.004.200
1,1998,3.149.124
2,1999,3.092.087
3,2000,3.238.022
4,2001,3.297.074
5,2002,3.343.711
6,2003,3.444.541
7,2004,3.496.238
8,2005,3.541.107
9,2006,3.589.728


In [62]:
df_altas_spain = df_altas_spain[df_altas_spain["Unnamed: 0"] != "Total"]

In [63]:
df_altas_spain

,Unnamed: 0,Número de altas
0,1997,3.004.200
1,1998,3.149.124
2,1999,3.092.087
3,2000,3.238.022
4,2001,3.297.074
5,2002,3.343.711
6,2003,3.444.541
7,2004,3.496.238
8,2005,3.541.107
9,2006,3.589.728


In [66]:
df_altas_spain = df_altas_spain.rename(columns={'Unnamed: 0': 'year'})

In [67]:
df_altas_spain

,year,Número de altas
0,1997,3.004.200
1,1998,3.149.124
2,1999,3.092.087
3,2000,3.238.022
4,2001,3.297.074
5,2002,3.343.711
6,2003,3.444.541
7,2004,3.496.238
8,2005,3.541.107
9,2006,3.589.728
